# SummerCourse Week2 HW
## Import library

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Define a custom dataset class

In [2]:
class MyDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        image = Image.fromarray(self.data_frame.iloc[idx, 1:].values.reshape(28, 28).astype(np.uint8))
        label = int(self.data_frame.iloc[idx, 0])
        if self.transform:
            image = self.transform(image)
        return image, label

## Data preprocessing

In [3]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=45),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

## Define dataset and dataloader

In [4]:
train_path = '/kaggle/input/fashionmnist/fashion-mnist_train.csv'
test_path = '/kaggle/input/fashionmnist/fashion-mnist_test.csv'

train_data = MyDataset(csv_file=train_path, transform=train_transform)
test_data = MyDataset(csv_file=test_path, transform=test_transform)
testLen = int(len(test_data) * 0.5)
valLen = len(test_data) - testLen
test_data, val_data = random_split(test_data, [testLen, valLen])

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
val_loader = DataLoader(val_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

## Define CNN model (initialize model, loss and optimizer)

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), weight_decay=0.001, lr=0.001)

## Train model

In [16]:
# Training section
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs):
    best_accuracy = 0.0
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct_train / total_train
        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

        # Evaluate on validation set every epoch
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

            val_loss = running_val_loss / len(val_loader.dataset)
            val_accuracy = correct_val / total_val

            if val_accuracy > best_accuracy:
                # Save model weights
                torch.save(model.state_dict(), 'best.pth')
                best_accuracy = val_accuracy

            print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
            
train_model(model, train_loader, val_loader, criterion, optimizer, 20)

Epoch 1/20, Training Loss: 1.0193, Training Accuracy: 0.6232
Validation Loss: 0.6394, Validation Accuracy: 0.7502
Epoch 2/20, Training Loss: 0.7509, Training Accuracy: 0.7206
Validation Loss: 0.5710, Validation Accuracy: 0.7720
Epoch 3/20, Training Loss: 0.6775, Training Accuracy: 0.7470
Validation Loss: 0.5397, Validation Accuracy: 0.7952
Epoch 4/20, Training Loss: 0.6322, Training Accuracy: 0.7672
Validation Loss: 0.4814, Validation Accuracy: 0.8182
Epoch 5/20, Training Loss: 0.6012, Training Accuracy: 0.7796
Validation Loss: 0.4605, Validation Accuracy: 0.8268
Epoch 6/20, Training Loss: 0.5751, Training Accuracy: 0.7933
Validation Loss: 0.4355, Validation Accuracy: 0.8376
Epoch 7/20, Training Loss: 0.5568, Training Accuracy: 0.7987
Validation Loss: 0.4217, Validation Accuracy: 0.8370
Epoch 8/20, Training Loss: 0.5364, Training Accuracy: 0.8064
Validation Loss: 0.4096, Validation Accuracy: 0.8470
Epoch 9/20, Training Loss: 0.5230, Training Accuracy: 0.8127
Validation Loss: 0.3992, Va

## Test model

In [17]:
# Testing section
def test_model(net, testloader):
    model.eval()
    running_test_loss = 0.0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_loss = running_test_loss / len(test_loader.dataset)
    test_accuracy = correct_test / total_test

    print(f'Testing Loss: {test_loss:.4f}, Testing Accuracy: {test_accuracy:.4f}')

# Load the best model for testing
model.load_state_dict(torch.load('best.pth'))
test_model(model, test_loader)

Testing Loss: 0.3118, Testing Accuracy: 0.8918
